# Assignment Week 5: Machine learning

#### Objective
Predict the outcomes in a data set using either Random Forest, Decision Tree or k-NN. 
I choose a dataset about Employee Attrition & Performance, the varaible I am going to predict is Attrition. 

#### Data
The Kaggle dataset: IBM HR Analytics Employee Attrition & Performance.
https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset predict the variable attrition.


In [73]:
import seaborn as sns
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split #We need this to split the data
from sklearn.neighbors import KNeighborsClassifier #the object class we need
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [74]:
df = pd.read_csv("employee.csv", sep=";")
df = df.dropna() #get rid of rows with empty cells
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


## Selecting variables

Attrition refers in this dataset to the loss of employees that the company faced for any certain reason; retirement, resignation or any medical issue, etc.

The goal of this analysis is finding out which factor determine attrition. In the dataset are a lot of useable variables, so let's see which variables have the highest correlation with Attrition. 

In [75]:
# show what variables we can work with
print(df.columns)

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')


In [76]:
df['Attrition'].replace('Yes',1, inplace=True)
df['Attrition'].replace('No',0, inplace=True)

# show the correlations of all variables with shares_log
df[df.columns[1:]].corr()['Attrition'][:]

Attrition                   1.000000
DailyRate                  -0.056652
DistanceFromHome            0.077924
Education                  -0.031373
EmployeeCount                    NaN
EmployeeNumber             -0.010577
EnvironmentSatisfaction    -0.103369
HourlyRate                 -0.006846
JobInvolvement             -0.130016
JobLevel                   -0.169105
JobSatisfaction            -0.103481
MonthlyIncome              -0.159840
MonthlyRate                 0.015170
NumCompaniesWorked          0.043494
PercentSalaryHike          -0.013478
PerformanceRating           0.002889
RelationshipSatisfaction   -0.045872
StandardHours                    NaN
StockOptionLevel           -0.137145
TotalWorkingYears          -0.171063
TrainingTimesLastYear      -0.059478
WorkLifeBalance            -0.063939
YearsAtCompany             -0.134392
YearsInCurrentRole         -0.160545
YearsSinceLastPromotion    -0.033019
YearsWithCurrManager       -0.156199
Name: Attrition, dtype: float64

#### The 5 variables with the highest correlation to Attrition 
- TotalWorkingYears          
- JobLevel                   
- YearsInCurrentRole         
- MonthlyIncome              
- YearsWithCurrManager       
            

In [77]:
df = df[['TotalWorkingYears', 'JobLevel', 'YearsInCurrentRole', 'MonthlyIncome', 'YearsWithCurrManager','Attrition']]
df['Attrition'].value_counts()

0    1233
1     237
Name: Attrition, dtype: int64

## k-nearest neighbor

k-nearest neighbor is one of the simplest algorithms in machine learning. k-NN works by finding the distances between a query and all the examples in the data, selecting the specified number examples closest to the query, then votes for the most frequent label.

In [78]:
# selecting x variables and y variable
X = df[['TotalWorkingYears', 'JobLevel', 'YearsInCurrentRole', 'MonthlyIncome', 'YearsWithCurrManager']]
y = df['Attrition']
# splitting into test and trainingdata
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=1) 
#show the head of the training set
X_train.head() 

,TotalWorkingYears,JobLevel,YearsInCurrentRole,MonthlyIncome,YearsWithCurrManager
99,17,2,2,2042,2
785,14,3,10,10322,1
918,31,5,10,19847,10
1335,7,2,2,3902,2
1182,4,2,2,4374,2


In [79]:
knn = KNeighborsClassifier() #create a KNN-classifier with 5 neighbors (default)
knn = knn.fit(X_train, y_train) #this fits the k-nearest neigbor model with the train data

## Model evaluation

In [80]:
knn.score(X_test, y_test) #calculate the fit on the *test* data

0.8072562358276644

Accuracy is 80.7%. An easy comparison is to compare with the best baseline guess: always guess "No". That would give us 1233 / (1233 + 237) = 83,88% (see value_counts above). So the model is not better than the baseline guess. Let's create a confusion matrix to evaluate precision and recall.

In [81]:
y_test_pred = knn.predict(X_test) #the predicted values
cm = confusion_matrix(y_test, y_test_pred) #creates a "confusion matrix"
conf_matrix = pd.DataFrame(cm, index=['No (actual)', 'Yes (actual)'], columns = ['No (predicted)', 'Yes (predicted)']) 
conf_matrix

,No (predicted),Yes (predicted)
No (actual),350,14
Yes (actual),71,6


In [82]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.83      0.96      0.89       364
           1       0.30      0.08      0.12        77

    accuracy                           0.81       441
   macro avg       0.57      0.52      0.51       441
weighted avg       0.74      0.81      0.76       441



- Precision for No = 83%
- Precision for Yes = 30%

- Recall for No = 96%
- Recall for Yes = 8%

The predictive performance is not good, so I am going to try different values for parameter k which hopefully leads to a more succesful result. 

## Parameter setting
I'll use a for-loop to do a simple parameter grid search. 


In [84]:
for i in range(1,11):
    knn_new = KNeighborsClassifier(n_neighbors = i) #make a new kNN model with i (1-10) neighbors
    knn_new = knn_new.fit(X_train, y_train) #fit new model on train data
    y_test_pred_new = knn_new.predict(X_test) #predict using new model, with test data
    print(f"With {i} neighbors the result is:")
    print(classification_report(y_test, y_test_pred_new)) #use a built-in function to print out accuracy, precision and recall

With 1 neighbors the result is:
              precision    recall  f1-score   support

           0       0.83      0.85      0.84       364
           1       0.21      0.19      0.20        77

    accuracy                           0.73       441
   macro avg       0.52      0.52      0.52       441
weighted avg       0.72      0.73      0.73       441

With 2 neighbors the result is:
              precision    recall  f1-score   support

           0       0.83      0.96      0.89       364
           1       0.24      0.06      0.10        77

    accuracy                           0.80       441
   macro avg       0.53      0.51      0.49       441
weighted avg       0.73      0.80      0.75       441

With 3 neighbors the result is:
              precision    recall  f1-score   support

           0       0.83      0.91      0.87       364
           1       0.23      0.13      0.17        77

    accuracy                           0.77       441
   macro avg       0.53      0.5